# Playground for Transformers!

### Attention is all you need 
(https://arxiv.org/abs/1706.03762)

### For software vulnerability detection GYM

This is a minimal example of this **CRAZY** idea!

### Note

* LSTM is working with:
    * Adam Learning rate = 0.01
    * Overfitted on 14 epoch. 100% training accuracy.
    * Worked on 1K dataset sample
    * Using the `hidden`n or `cell` output from LSTM. Not the `output`.
    * Bidrectional (2 layers)
* Transformer:
    * Still trying to find the right combination
    * Adam Learning rate 0.0001


### Dataset

Import

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import re
import torch.backends.cudnn as cudnn
import torchtext
import matplotlib.pyplot as plt
import time
import torchtext.vocab as vocab
import sklearn.metrics
import torch.nn.functional as F
import random
from torch.autograd import Variable
from torch import nn, optim
from torch.optim import SGD,Adam
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import confusion_matrix
from clang import cindex

In [2]:
seed = 1234
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
cudnn.benchmark = True
cudnn.enabled = True
print(device)

cpu


Load playset dataset

In [3]:
dataset = pd.read_pickle('playset(0.25.2).pickle')

View dataset

In [4]:
dataset

,functionSource,combine
93792,"go_file_opener_open (GOFileOpener const *fo, g...",False
79770,updatePathMap(bool left_level) {\n\tPoint from...,False
66999,interpret_tilde(const char* path) {\n stati...,False
44284,"checkVarExp(\n Absyn *node,\n Ta...",True
49515,will_have_skip_worktree(const struct cache_ent...,True
...,...,...
96701,"AVLTree_insert(AVLTree * tree, void * data)\n{...",False
67815,"remove_hook(const char *name, hookfn fn)\n{\n\...",False
88363,"output_def(dico_stream_t str, struct gcide_db ...",False
65929,getState(\n\t\tFLMUINT\t\tuiFieldID)\n\t{\n\t\...,False


In [5]:
dataset.describe(include='all')

,functionSource,combine
count,100000,100000
unique,100000,2
top,gst_alpha_chroma_key_ayuv_argb (const guint8 *...,True
freq,1,50000


In [6]:
dataset.functionSource[24492]

'CardPowerOff (reader* globalData, char socket) {\r\n    char cmd[4], ack;\r\n    int retVal, actual, retryTimes = 2;\r\n\r\n#ifdef ASE_DEBUG\r\n    syslog(LOG_INFO, "\\n CardPowerOff - Enter\\n");\r\n#endif\r\n\r\n    if ((retVal = cardCommandInit(globalData, socket, 1)))\r\n        return retVal;\r\n\r\n    cmd[0] = ASE_PACKET_TYPE(0x50, globalData->commandCounter, socket);\r\n    globalData->commandCounter++;\r\n    globalData->commandCounter %= 4;\r\n    cmd[1] = 0x21;\r\n    cmd[2] = 0x0;\r\n    cmd[3] = cmd[0] ^ cmd[1] ^ cmd[2];\r\n\r\n    do {\r\n        lock_mutex(globalData);\r\n        retVal = sendControlCommand(globalData, socket, cmd, 4, &ack, &actual, 0);\r\n        unlock_mutex(globalData);\r\n\r\n        retryTimes--;\r\n    } while (retVal != ASE_OK && retryTimes);\r\n\r\n    // if during the 3 tries the command failed, return an error status\r\n    if (retVal < 0) {\r\n        return retVal; \r\n    }\r\n\r\n    if (ack != 0x20) {\r\n        return parseStatus(ack); \

### Clang Tokenizer

In [6]:
## Check your library path for clang llvm
cindex.Config.set_library_path('/usr/lib/llvm-6.0/lib')
idx = cindex.Index.create()
global c
c=0
def clang_tokenizer(code):
    global c
    c+=1
    print('{0}'.format(c), end='\r')
    
    ## Remove code comments
    pat = re.compile(r'(/\*([^*]|(\*+[^*/]))*\*+/)|(//.*)')
    code = re.sub(pat,'',code)
    
    ## Tokkenize using clang
    tok = []
    tu = idx.parse('tmp.cpp',
                   args=[''],  
                   unsaved_files=[('tmp.cpp', code)],  
                   options=0)
    for t in tu.get_tokens(extent=tu.cursor.extent):
        tok.append(t.spelling)
    return(tok)

In [ ]:
dataset = dataset.functionSource.apply(clang_tokenizer)

#### Cleaner & Tokkenizer

In [3]:
global c
c=0
def my_tokenizer(code):
    global c
    c+=1
    print('{0}'.format(c), end='\r')
    
    pat = re.compile(r'(/\*([^*]|(\*+[^*/]))*\*+/)|(//.*)')
    code = re.sub(pat,'',code)
    
    ## Remove code comments
    #code = re.sub(r'(/\*([^*]|[\r\n]|(\*+([^*/]|[\r\n])))*\*+/)|(//.*)','',code)
    
    ## Remove newlines & tabs
    code = re.sub('(\n)|(\\\\n)|(\\\\)|(\\t)|(/)|(\\r)','',code)
    
    ## Mix split (characters and words)
    splitter = '\"(.*?)\"| +|(;)|(\()|(==)|(\))|(=)|(\+)|(\-)|(\[)|(\])|(<)|(>)|(\.)|({)'
    code = re.split(splitter,code)
    
    ## Remove None type
    code = list(filter(None, code))
    
    code = list(filter(str.strip, code))
    #code = " ".join(code)
    
    ## Return list of tokens
    return(code)

In [ ]:
dataset = dataset.functionSource.apply(my_tokenizer)

### Prepare data (in JSON)

In [8]:
## Change labels boolean to 1 and 0
dataset.iloc[:,1] = np.multiply(dataset.iloc[:,1],1)

In [9]:
## Change column name
dataset = dataset.rename(columns={'functionSource':'codes', 'combine':'label'})

In [10]:
false = dataset[dataset.iloc[:,1]==0]
true = dataset[dataset.iloc[:,1]==1]

In [11]:
## Split to train,test,valid
train = false[0:20000].append(true[0:1333])
test  = false[20000:22494].append(true[2000:2166])
valid = false[30000:32494].append(true[3000:3166])

## Shuffle
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)
valid = valid.sample(frac=1).reset_index(drop=True)

In [12]:
## Save to json
train.to_json('.data/train_ratio.json', orient='records',lines=True)
test.to_json('.data/test_ratio.json', orient='records',lines=True)
valid.to_json('.data/valid_ratio.json', orient='records',lines=True)

In [4]:
## Define the field

CODES = torchtext.data.Field(batch_first=True, tokenize=my_tokenizer)
LABEL = torchtext.data.LabelField(dtype=torch.long)
fields = {'codes': ('codes', CODES), 'label': ('label', LABEL)}

In [5]:
## Import ratio dataset 20K
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_ratio.json',
                                        validation = 'valid_ratio.json',
                                        test = 'test_ratio.json',
                                        format = 'json',
                                        fields = fields)

In [4]:
## Import the 1K data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_1k.json',
                                        validation = 'valid_1k.json',
                                        test = 'test_1k.json',
                                        format = 'json',
                                        fields = fields)

In [4]:
## Import the min(10k) data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_min.json',
                                        validation = 'valid_min.json',
                                        test = 'test_min.json',
                                        format = 'json',
                                        fields = fields)

In [4]:
## Import the 100K data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_100K.json',
                                        validation = 'valid_100k.json',
                                        test = 'test_100k.json',
                                        format = 'json',
                                        fields = fields)

In [15]:
## Testing
print(vars(valid_data[0]))

{'codes': ['serial8250_tx_chars', '(', 'struct', 'uart_8250_port', '*up', ')', '{', 'struct', 'uart_port', '*port', '=', '&up', '-', '>', 'port', ';', 'struct', 'circ_buf', '*xmit', '=', '&port', '-', '>', 'state', '-', '>', 'xmit', ';', 'int', 'count', ';', 'if', '(', 'port', '-', '>', 'x_char', ')', '{', 'serial_out', '(', 'up,', 'UART_TX,', 'port', '-', '>', 'x_char', ')', ';', 'port', '-', '>', 'icount', '.', 'tx', '+', '+', ';', 'port', '-', '>', 'x_char', '=', '0', ';', 'return', ';', '}if', '(', 'uart_tx_stopped', '(', 'port', ')', ')', '{', 'serial8250_stop_tx', '(', 'port', ')', ';', 'return', ';', '}if', '(', 'uart_circ_empty', '(', 'xmit', ')', ')', '{', '__stop_tx', '(', 'up', ')', ';', 'return', ';', '}count', '=', 'up', '-', '>', 'tx_loadsz', ';', 'do', '{', 'serial_out', '(', 'up,', 'UART_TX,', 'xmit', '-', '>', 'buf', '[', 'xmit', '-', '>', 'tail', ']', ')', ';', 'xmit', '-', '>', 'tail', '=', '(', 'xmit', '-', '>', 'tail', '+', '1', ')', '&', '(', 'UART_XMIT_SIZE', '-'

Doneeeeeeeeeeeeeeeee !

### Vocabulary-related preparation

In [6]:
##### Build the vocabulary

MAX_VOCAB_SIZE = 10000

CODES.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [7]:
print(f"Unique tokens in TEXT vocabulary: {len(CODES.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 2


In [8]:
## Most common word
print(CODES.vocab.freqs.most_common(100))

[('(', 304247), (')', 304234), (';', 282279), ('=', 138076), ('-', 125827), ('>', 119029), ('{', 71232), ('if', 56192), ('}', 45535), ('.', 42415), ('+', 37789), ('0', 37619), ('return', 31885), (']', 30465), ('[', 30445), (',', 24939), ('i', 24821), ('<', 23933), ('==', 20781), ('1', 19919), ('int', 18551), ('NULL', 17460), ('!', 13345), ('*', 13323), ('struct', 12115), ('const', 10376), ('char', 10091), ('else', 9572), ('&&', 8708), ('case', 8288), ('for', 7419), ('break', 7324), ('&', 6095), ('ret', 5850), ('||', 5534), ('p', 5018), ('unsigned', 4784), ('data', 4447), ('2', 4275), ('sizeof', 4085), ('}}', 3834), ('0,', 3794), ('len', 3715), ('j', 3594), ('void', 3575), ('n', 3564), ('s', 3525), ('size', 3291), ('while', 3255), ('|', 3215), ('x', 3203), ('c', 3160), ('goto', 2915), ('priv', 2914), ('result', 2803), ('name', 2781), ('}if', 2712), (':', 2700), ('type', 2692), ('false', 2604), ('this', 2585), ('FALSE', 2491), ('?', 2428), ('buf', 2389), ('NULL,', 2343), ("'", 2314), ('v

In [9]:
print(CODES.vocab.itos[:10])
print(LABEL.vocab.stoi)
print(CODES.vocab.stoi[CODES.pad_token])

['<unk>', '<pad>', '(', ')', ';', '=', '-', '>', '{', 'if']
defaultdict(None, {0: 0, 1: 1})
1


In [10]:
## place into iterators
train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = 64,
    sort = False)

### Prepare Word2Vec (Optional)

In [ ]:
corpus = pd.read_json('.data/train.json',orient='records',lines=True)

w2v = Word2Vec(corpus.codes, size=300, workers=16, sg=1, min_count=3)
w2v.save('.data/node_w2v_128')

In [10]:
w2v = Word2Vec.load('.data/node_w2v_128')
w2v.wv.vectors

array([[ 0.04268723, -0.01990928, -0.10372822, ...,  0.34928635,
        -0.24622028, -0.02363101],
       [ 0.06655177, -0.08706249, -0.11346684, ...,  0.2967248 ,
        -0.16500187, -0.10260527],
       [ 0.10059763, -0.0993171 , -0.14234892, ...,  0.3913037 ,
        -0.22237949,  0.02339004],
       ...,
       [-0.0031671 ,  0.01939397, -0.00094254, ..., -0.06062187,
        -0.0873417 ,  0.10190531],
       [-0.03719744,  0.02801778,  0.02174594, ..., -0.05577604,
        -0.07265704, -0.00079473],
       [ 0.01092949, -0.03061507, -0.045645  , ..., -0.06999503,
        -0.15679213,  0.11291362]], dtype=float32)

### Transformer class (with LSTM)

In [11]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer,self).__init__()
        
        self.embed = nn.Embedding(num_embeddings=10002,
                                  embedding_dim=128)
        self.encode_layer = nn.TransformerEncoderLayer(d_model=128,
                                                       nhead=8,
                                                       dim_feedforward=2048,
                                                       dropout=0.1,
                                                       activation='relu')
        self.trans_encoder = nn.TransformerEncoder(self.encode_layer,
                                                   num_layers=1)
        self.lstm1 = nn.LSTM(input_size=128,
                            hidden_size=128,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(128*2,2)
    
    def forward(self,x):
        x = self.embed(x)
        x = self.trans_encoder(x)
        output, (hidden, cell) = self.lstm1(x)
        x = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        x = self.fc1(x)
        return(x)

model = Transformer()
model.to(device)
print(model)

Transformer(
  (embed): Embedding(10002, 128)
  (encode_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): Linear(in_features=128, out_features=128, bias=True)
    )
    (linear1): Linear(in_features=128, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=128, bias=True)
    (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (trans_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_featur

### Load Word2Vec weights to embeddings (Optional)

In [ ]:
weights = torch.FloatTensor(w2v.wv.vectors)
weights = weights.to(device)
model.embed = model.embed.from_pretrained(weights)
#model.embed = model.embed.weight.data.copy_(weights)

### Parameters

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,126,274 trainable parameters


### Prepare accuracy function

In [13]:
def softmax_accuracy(probs,all_labels):
    def getClass(x):
        return(x.index(max(x)))
    
    all_labels = all_labels.tolist()
    probs = pd.Series(probs.tolist())
    all_predicted = probs.apply(getClass)
    all_predicted.reset_index(drop=True, inplace=True)
    vc = pd.value_counts(all_predicted == all_labels)
    try:
        acc = vc[1]/len(all_labels)
    except:
        if(vc.index[0]==False):
            acc = 0
        else:
            acc = 1
    return(acc)

### Loss functions

In [14]:
## Define optimizer
#optimizer = SGD(model.parameters(), lr = 0.01)
#optimizer = Adam(model.parameters(), lr=0.0001, weight_decay=0.01)
optimizer = Adam(model.parameters(), lr=0.0005)


## Define loss function
#criterion = nn.BCELoss().to(device) ## Sigmoid activation function
#criterion = nn.NLLLoss().to(device) ### Log_softmax activation

weights = torch.tensor([1.0, 6.0])
criterion = nn.CrossEntropyLoss(weight=weights).to(device) ## No activation function bcs softmax included

### Training

In [15]:

epochs=100
for e in range(epochs):
    running_acc = 0
    running_loss = 0
    timer = time.time()
    model.train()

    for batch in train_iterator:
        batch.codes, batch.label = batch.codes.to(device), batch.label.to(device)
        optimizer.zero_grad()
        output = model(batch.codes)
        loss = criterion(output, batch.label)
        loss.backward()
        optimizer.step()
        acc = softmax_accuracy(output,batch.label)
        running_acc += acc.item()
        running_loss += loss.item()
    else:
        with torch.no_grad():
            model.eval()
            running_acc_val = 0
            running_loss_val = 0
            for batch in valid_iterator:
                batch.codes, batch.label = batch.codes.to(device), batch.label.to(device)
                output_val = model(batch.codes)
                loss_val = criterion(output_val,batch.label)
                acc_val = softmax_accuracy(output_val,batch.label)
                running_acc_val += acc_val.item()
                running_loss_val += loss_val.item()
        
        print("Epoch {} - Training acc: {:.6f} -Training loss: {:.6f} - Val acc: {:.6f} - Val loss: {:.6f} - Time: {:.4f}s".format(e+1, running_acc/len(train_iterator), running_loss/len(train_iterator), running_acc_val/len(valid_iterator), running_loss_val/len(valid_iterator), (time.time()-timer)))

Epoch 1 - Training acc: 0.904512 -Training loss: 0.554553 - Val acc: 0.852348 - Val loss: 0.519172 - Time: 179.1126s
Epoch 2 - Training acc: 0.872844 -Training loss: 0.504077 - Val acc: 0.891493 - Val loss: 0.510825 - Time: 169.8695s
Epoch 3 - Training acc: 0.878130 -Training loss: 0.484272 - Val acc: 0.890377 - Val loss: 0.475982 - Time: 164.8433s
Epoch 4 - Training acc: 0.878034 -Training loss: 0.437897 - Val acc: 0.859706 - Val loss: 0.447219 - Time: 164.4663s
Epoch 5 - Training acc: 0.889498 -Training loss: 0.405196 - Val acc: 0.884259 - Val loss: 0.466898 - Time: 163.3034s
Epoch 6 - Training acc: 0.896508 -Training loss: 0.379013 - Val acc: 0.914021 - Val loss: 0.473878 - Time: 163.7093s
Epoch 7 - Training acc: 0.899792 -Training loss: 0.357238 - Val acc: 0.899140 - Val loss: 0.486168 - Time: 164.1801s
Epoch 8 - Training acc: 0.899271 -Training loss: 0.339980 - Val acc: 0.900174 - Val loss: 0.471316 - Time: 164.9214s
Epoch 9 - Training acc: 0.911621 -Training loss: 0.311300 - Val 

In [ ]:
### Evaluate on Training set

import sklearn.metrics
from sklearn.metrics import confusion_matrix

model.eval()
with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in train_iterator:
        batch.codes, batch.label = batch.codes.to(device), batch.label.to(device)
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = softmax_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += output_test.tolist()
        all_labels += batch.label.tolist()


print('Train acc: ',running_acc_test/len(train_iterator))
print('Train loss: ',running_loss_test/len(train_iterator))


def getClass(x):
    return(x.index(max(x)))

probs = pd.Series(all_pred)
all_predicted = probs.apply(getClass)
all_predicted.reset_index(drop=True, inplace=True)
vc = pd.value_counts(all_predicted == all_labels)

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_predicted)
print('Confusion matrix: \n',confusion)

### Testing

In [17]:
### Evaluate on Testing set

model.eval()
with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in test_iterator:
        batch.codes, batch.label = batch.codes.to(device), batch.label.to(device)
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = softmax_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += output_test.tolist()
        all_labels += batch.label.tolist()


print('Test acc: ',running_acc_test/len(test_iterator))
print('Test loss: ',running_loss_test/len(test_iterator))


def getClass(x):
    return(x.index(max(x)))

probs = pd.Series(all_pred)
all_predicted = probs.apply(getClass)
all_predicted.reset_index(drop=True, inplace=True)
vc = pd.value_counts(all_predicted == all_labels)

probs2=[]
for x in probs:
    probs2.append(x[1])

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_predicted)
print('Confusion matrix: \n',confusion)


tn, fp, fn, tp = confusion.ravel()
print('\nTP:',tp)
print('FP:',fp)
print('TN:',tn)
print('FN:',fn)

## Performance measure
print('\nAccuracy: '+ str(sklearn.metrics.accuracy_score(y_true=all_labels, y_pred=all_predicted)))
print('Precision: '+ str(sklearn.metrics.precision_score(y_true=all_labels, y_pred=all_predicted)))
print('Recall: '+ str(sklearn.metrics.recall_score(y_true=all_labels, y_pred=all_predicted)))
print('F-measure: '+ str(sklearn.metrics.f1_score(y_true=all_labels, y_pred=all_predicted)))
print('Precision-Recall AUC: '+ str(sklearn.metrics.average_precision_score(y_true=all_labels, y_score=probs2)))
print('AUC: '+ str(sklearn.metrics.roc_auc_score(y_true=all_labels, y_score=probs2)))
print('MCC: '+ str(sklearn.metrics.matthews_corrcoef(y_true=all_labels, y_pred=all_predicted)))

Test acc:  0.8771908068783068
Test loss:  0.4461185729929379
Confusion matrix: 
 [[2249  245]
 [  82   84]]

TP: 84
FP: 245
TN: 2249
FN: 82

Accuracy: 0.8770676691729323
Precision: 0.2553191489361702
Recall: 0.5060240963855421
F-measure: 0.3393939393939393
Precision-Recall AUC: 0.2500610212460307
AUC: 0.8273470787721858
MCC: 0.2996180523004313


# RESULTS (Testing)

Weights : 1:3 (400)
val loss: 0.396747
Test acc:  0.9320849867724867
Test loss:  0.39700411295606974
Confusion matrix: 
 [[2471   23]
 [ 158    8]]

TP: 8
FP: 23
TN: 2471
FN: 158

Accuracy: 0.9319548872180451
Precision: 0.25806451612903225
Recall: 0.04819277108433735
F-measure: 0.08121827411167513
Precision-Recall AUC: 0.17136589216762943
AUC: 0.717964077641762
MCC: 0.08783419878226471


Weights: 1:15 (400)
Test acc:  0.8045221560846562
Test loss:  0.5900836005097344
Confusion matrix: 
 [[2050  444]
 [  76   90]]

TP: 90
FP: 444
TN: 2050
FN: 76

Accuracy: 0.8045112781954887
Precision: 0.16853932584269662
Recall: 0.5421686746987951
F-measure: 0.2571428571428571
Precision-Recall AUC: 0.17323523591325418
AUC: 0.7548574409909082
MCC: 0.21989739199806438


Weights: 1:5 (400)
Test acc:  0.8738425925925926
Test loss:  0.4650661984369868
Confusion matrix: 
 [[2252  242]
 [  94   72]]

TP: 72
FP: 242
TN: 2252
FN: 94

Accuracy: 0.8736842105263158
Precision: 0.22929936305732485
Recall: 0.43373493975903615
F-measure: 0.3
Precision-Recall AUC: 0.1893410043394382
AUC: 0.7772436981285205
MCC: 0.2524173571221226


Weights: 1:5 (400) - no weight decay- lr 0.0005
Test acc:  0.908068783068783
Test loss:  0.4351889540751775
Confusion matrix: 
 [[2352  142]
 [ 102   64]]

TP: 64
FP: 142
TN: 2352
FN: 102

Accuracy: 0.9082706766917293
Precision: 0.3106796116504854
Recall: 0.3855421686746988
F-measure: 0.3440860215053763
Precision-Recall AUC: 0.2703683299643296
AUC: 0.805646805344876
MCC: 0.2973762185965168


Weights: 1:5 (128)- no weight decay- lr 0.0005
Test acc:  0.8882688492063492
Test loss:  0.5016442422001135
Confusion matrix: 
 [[2271  223]
 [  75   91]]

TP: 91
FP: 223
TN: 2271
FN: 75

Accuracy: 0.8879699248120301
Precision: 0.2898089171974522
Recall: 0.5481927710843374
F-measure: 0.3791666666666667
Precision-Recall AUC: 0.28130664351504053
AUC: 0.8167771325880908
MCC: 0.3439348556798436

Weights: 1:5 (128) - lr 0.001
Test acc:  0.8771908068783068
Test loss:  0.4461185729929379
Confusion matrix: 
 [[2249  245]
 [  82   84]]

TP: 84
FP: 245
TN: 2249
FN: 82

Accuracy: 0.8770676691729323
Precision: 0.2553191489361702
Recall: 0.5060240963855421
F-measure: 0.3393939393939393
Precision-Recall AUC: 0.2500610212460307
AUC: 0.8273470787721858
MCC: 0.2996180523004313

